In [6]:
import pandas as pd
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display

# 读取数据
df = pd.read_csv('../../../Dataset/tracking_week_1.csv')

In [7]:
# 筛选特定的gameId和playId
def filter_data(df, game_id, play_id):
    return df[(df['gameId'] == game_id) & (df['playId'] == play_id)]

# # 示例的gameId和playId
# game_id_example = 2022090800
# play_id_example = 56
# 
# # 筛选数据
# filtered_df = filter_data(df, game_id_example, play_id_example)
# filtered_df.head()
# # filtered_df.to_csv('filtered_df.csv')

In [8]:
# 绘制球场
import plotly.graph_objects as go

def draw_detailed_football_field():
    field_shapes = []
    field_annotations = []

    # field_shapes.append(go.layout.Shape(type="rect", x0=0, y0=0, x1=120, y1=53.3, line=dict(color="white"), fillcolor='green'))
    # field_shapes.append(go.layout.Shape(type="rect", x0=0, y0=0, x1=10, y1=53.3, line=dict(color="white"), fillcolor='darkgreen'))
    # field_shapes.append(go.layout.Shape(type="rect", x0=110, y0=0, x1=120, y1=53.3, line=dict(color="white"), fillcolor='darkgreen'))
    
    field_shapes.append(go.layout.Shape(type="rect", x0=0, y0=0, x1=120, y1=53.3, line=dict(color="white")))
    field_shapes.append(go.layout.Shape(type="rect", x0=0, y0=0, x1=10, y1=53.3, line=dict(color="white")))
    field_shapes.append(go.layout.Shape(type="rect", x0=110, y0=0, x1=120, y1=53.3, line=dict(color="white")))

    # 增加码线和小标记线
    for line in range(10, 110, 5):
        # 码线
        field_shapes.append(go.layout.Shape(type="line", x0=line, y0=0, x1=line, y1=53.3, line=dict(color="white")))
        # 小标记线
        gate = 26.65
        dis = 31 - gate
        base = 1
        for mark in range(line + 1, line + 5):
            field_shapes.append(go.layout.Shape(type="line", x0=mark, y0=gate+dis, x1=mark, y1=gate+dis+1.5, line=dict(color="white", width=3)))
            field_shapes.append(go.layout.Shape(type="line", x0=mark, y0=gate-dis, x1=mark, y1=gate-dis-1.5, line=dict(color="white", width=3)))
            field_shapes.append(go.layout.Shape(type="line", x0=mark, y0=base, x1=mark, y1=base+1.5, line=dict(color="white", width=3)))
            field_shapes.append(go.layout.Shape(type="line", x0=mark, y0=53.3-base, x1=mark, y1=53.3-base-1.5, line=dict(color="white", width=3)))

    # 码数
    for i in range(10, 100, 10):
        number = str(i) if i <= 50 else str(100 - i)
        field_annotations.append(go.layout.Annotation(x=i+10, y=8, text=number, showarrow=False, font=dict(size=30, family="Din", color="white")))
        field_annotations.append(go.layout.Annotation(x=i+10, y=44, text=number, showarrow=False, font=dict(size=30, family="Din", color="white")))
        
        # 进攻方向
        if i < 50:
            field_annotations.append(go.layout.Annotation(x=i+8, y=8, text="◀", showarrow=False, font=dict(size=10, color="white")))
            field_annotations.append(go.layout.Annotation(x=i+8, y=44, text="◀", showarrow=False, font=dict(size=10, color="white")))
        elif i > 50:
            field_annotations.append(go.layout.Annotation(x=i+12, y=8, text="▶", showarrow=False, font=dict(size=10, color="white")))
            field_annotations.append(go.layout.Annotation(x=i+12, y=44, text="▶", showarrow=False, font=dict(size=10, color="white")))
            
    # 球门
    field_annotations.append(go.layout.Annotation(x=0, y=26.65, text="| |", showarrow=False, font=dict(size=30, color="white")))
    field_annotations.append(go.layout.Annotation(x=120, y=26.65, text="| |", showarrow=False, font=dict(size=30, color="white")))

    return field_shapes, field_annotations

In [9]:
# 绘制动画
def create_animation(filtered_df, initial_frame):
    # 获取详细的足球场地形状和注释
    field_shapes, field_annotations = draw_detailed_football_field()

    # 确定两支球队的名称
    team_colors = {}
    team_names = filtered_df['club'].unique()
    
    for team in team_names:
        if team != 'football':
            if not team_colors:  
                team_colors[team] = 'blue'
            else:  
                team_colors[team] = 'red'
    
    team_colors['football'] = 'green'

    # 定义图形布局
    fig = go.Figure(
        layout=go.Layout(
            shapes=field_shapes,  # 添加场地形状
            annotations=field_annotations,
            xaxis=dict(range=[0, 120], autorange=False, showgrid=False, showticklabels=False),
            yaxis=dict(range=[0, 53.3], autorange=False, showgrid=False, showticklabels=False),
            updatemenus=[dict(
                type="buttons",
                showactive=False,
                buttons=[dict(
                    label="Play",
                    method="animate",
                    args=[None, dict(frame=dict(duration=50, redraw=True), fromcurrent=True)]
                )]
            )],
            sliders=[dict(
                steps=[dict(method='animate', args=[[f'{k}'],
                                                    dict(mode='immediate', frame=dict(duration=50, redraw=True), fromcurrent=True)],
                     label=f'{k}')
                       for k in filtered_df['frameId'].unique()],
                transition=dict(duration=0),
                x=0,
                y=0,
                currentvalue=dict(font=dict(size=12), prefix='Frame: ', visible=True),
                len=1.0
            )]
        )
    )

    # 为每个帧创建数据集，区分主队和客队
    frames = []
    for frame_number, frame_data in filtered_df.groupby('frameId'):
        frame_traces = []
        for club, data in frame_data.groupby('club'):
            color = team_colors[club]
            frame_traces.append(go.Scatter(
                x=data['x'], y=data['y'], mode='markers+text',
                text=data['nflId'], name=club,
                marker=dict(color=color, size=10),
                textfont=dict(color=color),
                textposition='bottom center'
            ))
        frames.append(go.Frame(data=frame_traces, name=str(frame_number)))

    # 将帧添加到图形中
    fig.frames = frames

    # 添加初始位置的球员散点图
    for club, data in initial_frame.groupby('club'):
        color = team_colors[club]
        fig.add_trace(go.Scatter(
            x=data['x'], y=data['y'], mode='markers+text',
            text=data['nflId'], name=club,
            marker=dict(color=color, size=10),
            textfont=dict(color=color),
            textposition='bottom center'
        ))

    # 设置动画
    fig.update_layout(transition=dict(duration=50), title="一次进攻的动态图")

    return fig



In [10]:
# 文本框用于输入 gameId 和 playId
game_id_input = widgets.Text(
    value='2022090800',  # 示例的gameId
    description='Game ID:',
    disabled=False
)

play_id_input = widgets.Text(
    value='56',  # 示例的playId
    description='Play ID:',
    disabled=False
)

# 按钮用于触发可视化
button = widgets.Button(description="Generate Visualization")

# 输出区域
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        # 获取用户输入的 gameId 和 playId
        game_id = int(game_id_input.value)
        play_id = int(play_id_input.value)
        
        # 筛选数据
        filtered_df = filter_data(df, game_id, play_id)
        initial_frame = filtered_df[filtered_df['frameId'] == filtered_df['frameId'].min()]
        
        # 生成并显示图形
        fig = create_animation(filtered_df, initial_frame)
        fig.show()

button.on_click(on_button_clicked)

# 显示界面
display(game_id_input, play_id_input, button, output)

Text(value='2022090800', description='Game ID:')

Text(value='56', description='Play ID:')

Button(description='Generate Visualization', style=ButtonStyle())

Output()